In [1]:
import pandas 
import numpy as np
import matplotlib.pylab as pl
import tensorflow as tf

from PIL import Image
import random

import math

data = pandas.read_csv('./annotations.csv',header = None, 
                       names = ['FileName','StartPoint_X','StartPoint_Y','EndPoint_X','EndPoint_Y'])



In [2]:
level_0 = data[data['FileName'].str.contains('0_level')]

#level_0

In [3]:
#get FileName array

array_FileName = level_0.FileName.values.reshape(-1,1)

#get StartPoint_X, StartPoint_Y,EndPoint_X,EndPoint_Y array

array_Point = level_0.drop(columns=['FileName'], inplace=False).values


array_Point.shape




(8471, 4)

In [ ]:
x_dimension = 160
y_dimension = 160

size = x_dimension,y_dimension
location_x = []
location_y = []
for i in range(0,len(array_FileName)):
    img = Image.open('./level0_train/'+array_FileName[i][0]+'.png').convert('L')
    img.thumbnail(size)
    img_np = np.array(img)
    #The first step find the location x and location y
    location = np.where(img_np>0)
    
    location_x.append(min(location[0].tolist()))
    location_x.append(max(location[0].tolist()))
    location_y.append(min(location[1].tolist()))
    location_y.append(max(location[0].tolist()))
    
#print(location)

location_x_min = min(location_x)
location_x_max = max(location_x)
location_y_min = min(location_y)
location_y_max = max(location_y)
#print(location_x)
#print(location_y)
print(location_x_min)#47
print(location_x_max)#116
print(location_y_min)#47
print(location_y_max)#116

img = Image.open('./level0_train/'+array_FileName[0][0]+'.png').convert('L')
img.thumbnail(size)
img_np = np.array(img)

img_np = np.delete(img_np,range(0,location_x_min),axis = 0)
print(img_np.shape)
img_np = np.delete(img_np,range((location_x_max-location_x_min),(x_dimension-location_x_min)),axis = 0)#
print(img_np.shape)
img_np = np.delete(img_np,range(0,location_y_min),axis = 1)
print(img_np.shape)
img_np = np.delete(img_np,range((location_y_max-location_y_min),(y_dimension-location_y_min)),axis = 1)
print(img_np.shape)

Image.fromarray(np.uint8(img_np))
pl.imshow(Image.fromarray(np.uint8(img_np)))
pl.show()



In [18]:
# image processing

holdout = random.sample(range(0,8471),800)

#array_imageName = array_FileName[holdout]


size = 160,160

location_x_min = 47
location_x_max = 116
location_y_min = 47
location_y_max = 116

# grey image
#img = Image.open('./level0/'+array_imageName[0][0]+'.png').convert('L')

#img.thumbnail(size)

#image_array = np.array(img).reshape(-1,1)






image = []

for i in range(0,len(array_FileName)):
    img = Image.open('./level0_train/'+array_FileName[i][0]+'.png').convert('L')
    
    img.thumbnail(size)
    #pl.imshow(img)
    #pl.show()
    #print (np.array(img))
    img_np = np.array(img)

    img_np = np.delete(img_np,range(0,location_x_min),axis = 0)
    #print(img_np.shape)
    img_np = np.delete(img_np,range((location_x_max-location_x_min),(160-location_x_min)),axis = 0)#
    #print(img_np.shape)
    img_np = np.delete(img_np,range(0,location_y_min),axis = 1)
    #print(img_np.shape)
    img_np = np.delete(img_np,range((location_y_max-location_y_min),(160-location_y_min)),axis = 1)
    #print(img_np.shape)
    
    
    if i==0 :
        
        image_array = img_np.reshape(-1,1)
    else:
        image_array = np.c_[image_array,img_np.reshape(-1,1)]
    
    
    #print(image_array.shape)
np.savetxt('./image.csv', image_array.T, delimiter = ',')
    
#level0_test_input = image_array.T[holdout]
#level0_test_output = array_Point[holdout]

#level0_train_input = np.delete(image_array.T, holdout, 0)
#level0_train_output = np.delete(array_Point, holdout, 0)




#print(test_train_input)


#image_array = image_array.reshape(-1,1)
'''
print(level0_train_input.shape)
print(level0_train_output.shape)
print(level0_test_input.shape)
print(level0_test_output.shape)
'''



#np.savetxt('new.csv', image_array, delimiter = ' ')



'\nprint(level0_train_input.shape)\nprint(level0_train_output.shape)\nprint(level0_test_input.shape)\nprint(level0_test_output.shape)\n'

In [5]:
level0_test_input.astype(np.float32)
level0_train_input.astype(np.float32)
#test_train_output.dtype

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
level0_train_output.astype(np.float32)
level0_test_output.astype(np.float32)


array([[270., 286., 322., 361.],
       [247., 209., 230., 152.],
       [260., 262., 258., 261.],
       ...,
       [274., 196., 301.,  95.],
       [225., 198., 165.,  93.],
       [219., 251., 118., 226.]], dtype=float32)

In [7]:
#design network

tf.reset_default_graph() # 这个可以不用细究，是为了防止重复定义报错

# 给X、Y定义placeholder，要指定数据类型、形状：
X = tf.placeholder(dtype=tf.float32,shape=[None,4761],name='Input')
Y = tf.placeholder(dtype=tf.float32,shape=[None,4],name='Output')

# 定义各个参数：
'''
W1 = tf.get_variable('W1',[4900,128],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b1',[128],initializer=tf.zeros_initializer())
W2 = tf.get_variable('W2',[128,64],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('b2',[64],initializer=tf.zeros_initializer())
W3 = tf.get_variable('W3',[64,4],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable('b3',[4],initializer=tf.zeros_initializer())
'''
W1 = tf.get_variable('W1',[4761,2048],initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable('b1',[2048],initializer=tf.zeros_initializer())
W2 = tf.get_variable('W2',[2048,512],initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable('b2',[512],initializer=tf.zeros_initializer())
W3 = tf.get_variable('W3',[512,64],initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable('b3',[64],initializer=tf.zeros_initializer())
W4 = tf.get_variable('W4',[64,4],initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable('b4',[4],initializer=tf.zeros_initializer())





In [8]:

    A1 = tf.nn.relu(tf.matmul(X,W1)+b1,name='A1')
    A2 = tf.nn.relu(tf.matmul(A1,W2)+b2,name='A2')
    A3 = tf.nn.relu(tf.matmul(A2,W3)+b3,name='A3')
    Z4 = tf.matmul(A3,W4)+b4








In [9]:
#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z3,labels=Y))
cost = tf.reduce_mean(tf.square(Y - Z4))




In [10]:
trainer = tf.train.AdamOptimizer().minimize(cost)
#learning_rate = tf.placeholder(tf.float32, shape=[])

#trainer = tf.train.GradientDescentOptimizer (0.00001).minimize (cost)
#init = tf.global_variables_initializer ()
#correct_prediction = tf.equal (tf.argmax (Y, 1), tf.argmax (Z3, 1))
#accuracy = tf.reduce_mean (tf.cast (correct_prediction, 'float'))



In [15]:
with tf.Session() as sess:
    # 首先给所有的变量都初始化（不用管什么意思，反正是一句必须的话）：
    sess.run(tf.global_variables_initializer())

    # 定义一个costs列表，来装迭代过程中的cost，从而好画图分析模型训练进展
    costs = []
    costs_100 = []

    # 指定迭代次数：
    for it in range(2500):
        # 这里我们可以使用mnist自带的一个函数train.next_batch，可以方便地取出一个个地小数据集，从而可以加快我们的训练：
        #X_batch,Y_batch = mnist.train.next_batch(batch_size=64)

        # 我们最终需要的是trainer跑起来，并获得cost，所以我们run trainer和cost，同时要把X、Y给feed进去：
        _,batch_cost = sess.run([trainer,cost],feed_dict={X:level0_train_input,Y:level0_train_output})
        costs.append(batch_cost)

        # 每100个迭代就打印一次cost：
        if it%100 == 0:
            costs_100.append(batch_cost)
            print('iteration%d ,batch_cost: '%it,batch_cost)

    # 训练完成，我们来分别看看来训练集和测试集上的准确率：
    predictions = tf.equal(tf.argmax(tf.transpose(Z4)),tf.argmax(tf.transpose(Y)))
    accuracy = tf.reduce_mean(tf.cast(predictions,'float'))
    cost_test = sess.run(cost,feed_dict = {X:level0_train_input,Y:level0_train_output})
    print (cost_test)
    print("Training set accuracy: ",sess.run(accuracy,feed_dict={X:level0_train_input,Y:level0_train_output}))



iteration0 ,batch_cost:  99775.12
iteration100 ,batch_cost:  5317.133
iteration200 ,batch_cost:  5128.149
iteration300 ,batch_cost:  5011.426
iteration400 ,batch_cost:  4877.836
iteration500 ,batch_cost:  4831.427
iteration600 ,batch_cost:  4759.379
iteration700 ,batch_cost:  4743.875
iteration800 ,batch_cost:  4687.3857
iteration900 ,batch_cost:  4807.637
iteration1000 ,batch_cost:  4631.961
iteration1100 ,batch_cost:  4593.751
iteration1200 ,batch_cost:  6153.099
iteration1300 ,batch_cost:  4557.988
iteration1400 ,batch_cost:  4509.464
iteration1500 ,batch_cost:  4636.908
iteration1600 ,batch_cost:  4570.2817
iteration1700 ,batch_cost:  4526.543
iteration1800 ,batch_cost:  4482.5625
iteration1900 ,batch_cost:  4435.187
iteration2000 ,batch_cost:  4382.565
iteration2100 ,batch_cost:  4341.3804
iteration2200 ,batch_cost:  4288.044
iteration2300 ,batch_cost:  4275.228
iteration2400 ,batch_cost:  4285.473
5173.4097


In [12]:


sess.close()
